# RAG

Conforme explicado anteriormente, o RAG é um conjunto de técnicas que visam expandir o contexto de uma LLM, adaptando-a a problemas e contextos específicos. Muitas implementações estão feitas na biblioteca [Langchain](https://www.langchain.com/) (leia a documentação!) e partiremos dela.

In [ ]:
# Instalação de bibliotecas
!pip install langchain chromadb langchain-community ibm-watsonx-ai langchain-ibm

# Credenciais

Todo o setup está sendo feito com base no arquivo `.env` que está no repositório. Essa configuração inicial pode ser revisada com mais calma [aqui](https://dataplatform.cloud.ibm.com/exchange/public/entry/view/c3dbf23a-9a56-4c4b-8ce5-5707828fc981?context=wx)

In [1]:
from dotenv import load_dotenv, find_dotenv
import os


_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.environ.get('WATSONX_APIKEY'),
}

# Teste do watsonx

Agora, faremos o setup dos parâmetros. Eles podem ser observados no Prompt Lab. O mais importante é o `DECODING_METHOD`, que aceita as opções greedy e sample:

- Greedy: sempre pega o token com a maior probabilidade, tendendo a ser mais **estável**
- Sampling: insere aleatoriedade no processo de geração, dando uma chance maior para que tokens com a menor probabilidade sejam escolhidos

`MAX_NEW_TOKENS`: quantidade máxima de novos tokens a serem gerados

`MIN_NEW_TOKENS`: quantidade mínima de novos tokens a serem gerados

A partir da escolha do método de sampling, outras variáveis passam a ser utilizadas:

`TEMPERATURE`: pode ser entendida como a "criatividade" da geração. Quanto maior a temperatura, maior a probabilidade de escolher tokens com uma menor probabilidade. Varia de 0 a 2.

`TOP_K` e `TOP_P`: controlam como os tokens são selecionados. Os valores típicos são de 50 e 1, respectivamente.

In [74]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 100,
    GenParams.TOP_P: 1,
}

Agora, vamos fazer a instanciação do modelo em si. Primeiramente, vamos listar os modelos disponíveis, para consulta e lembrando que os mesmos podem ser testados no Prompt Lab de forma rápida e prática.

In [81]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2']


In [82]:
from ibm_watson_machine_learning.foundation_models import Model
import os

flan_t5_model = Model(
    model_id=ModelTypes.FLAN_T5_XXL,
    credentials=credentials, # Credenciais definidas no início do notebook
    project_id=os.environ.get('WATSONX_PROJECT_ID'), # ID do projeto no CP4D
    params=parameters)

# flan_t5_ul2_model = Model(
#     model_id="google/flan-t5-ul2",
#     credentials=credentials,
#     project_id=os.environ.get('WATSONX_PROJECT_ID'))

O LangChain funciona com base na ideia da montagem de cadeias de informação. Com isso, vamos testar o modelo carregado a partir do watsonx com a seguinte estrutura:

1. PromptTemplate: um esqueleto de prompt que será enviado para o LLM
2. SimpleSequentialChain: basicamente vai construir a entrada a partir do template e da entrada

Para um teste simples, vamos passar um review para que o modelo classifique o mesmo como positivo ou negativo:

In [60]:
from langchain import PromptTemplate


template = """Based on the following review, categorize the sentiment as 'positive' or 'negative':
{review} 
Sentiment: "
"""

# Aqui, 'review' é a entrada (que será passada mais adiante)
prompt_test = PromptTemplate(
    input_variables=["review"],
    template=template,
)

In [61]:
from langchain.chains import LLMChain


prompt_to_flan_t5 = LLMChain(llm=flan_t5_model.to_langchain(), # O método '.to_langchain()' é usado para transformar o modelo para um formato reconhecido pelas demais funções do langchain
                             prompt=prompt_test)

In [62]:
from langchain.chains import SimpleSequentialChain


# Montagem da chain
qa = SimpleSequentialChain(chains=[prompt_to_flan_t5], verbose=True)

In [63]:
review = """Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!"""

qa.run(review)



> Entering new SimpleSequentialChain chain...
positive

> Finished chain.


'positive'

# Solução de RAG em cima de uma base de conhecimento

[Referência](https://python.langchain.com/docs/use_cases/question_answering/)

**Como criar um sistema que consiga construir respostas com base em um contexto específico?**

Para exemplificar o problema, vamos partir de um conjunto de artigos do IEEE. Todos tem um esqueleto básico, mas possuem formatações completamente diferentes entre si. Após extrair os textos desses artigos, vamos armazená-los em uma base de alguma forma para construir a resposta com base neles.

No entanto, não é possível colocar todos os artigos dentro da janela de contexto/prompt do LLM. Com isso, um processo de extração das passagens mais importantes também deve ser feito, de forma que o LLM receba a pergunta e as parcelas mais importantes da base de conhecimento para gerar sua resposta. A estrutura ficará dessa forma:

<img src="images/rag_retrieval_generation.png" height="400px">

1. A Pergunta entra na base, onde um processo de extração encontra os fragmentos mais pertinentes para responder a pergunta;
2. Os fragmentos e a questão são inseridas no prompt;
3. O prompt é enviado para o LLM e uma resposta é dada.

**Como construir esse banco de informações?**

Esse banco de informações receberá (no nosso caso) PDFs, mas o langchain possui [métodos nativos](https://python.langchain.com/docs/modules/data_connection/document_loaders/) para CSVs, HTML, JSON, Markdown, PDF e outros.

Esses documentos são segmentados em parcelas menores (e configuráveis). Dessa forma, o fatiamento permite que os documentos estejam separados em parcelas de mesmo tamanho.

Após esse _split_, os documentos são vetorizados. Assim, eles podem ser comparados matematicamente entre si e com a pergunta por meio de operações semelhantes a operações euclidianas.

Por fim, esses vetores são armazenados em uma base de dados chamada de **Vector store** ou **vector DB**.

<img src="images/vector_db.png" height="400px">

## Carregamento dos dados no VectorDB

In [9]:
from glob import glob
from os.path import join
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader


# Onde os arquivos estão salvos
files_dir = 'data'

# Use a função glob para listar todos os arquivos PDF no diretório
pdf_files = glob(join(files_dir, '*.pdf'))

docs=[]

for arquivo in tqdm(pdf_files):
    print(arquivo)

    loader = PyPDFLoader(arquivo)
    docs.extend(loader.load())

  0%|          | 0/34 [00:00<?, ?it/s]

data\35-GHz Barium Hexaferrite or PDMS Composite-Based Millimeter-Wave Circulators for 5G Applications.pdf


  3%|▎         | 1/34 [00:00<00:24,  1.36it/s]

data\A comprehensive survey on machine learning for networking - Evolution, applications and research opportunities.pdf


  6%|▌         | 2/34 [00:04<01:14,  2.32s/it]

...

 91%|█████████ | 31/34 [01:14<00:02,  1.03it/s]

data\Time4 - Time for SDN.pdf


 97%|█████████▋| 33/34 [01:16<00:01,  1.04s/it]

data\Towards Supporting Intelligence in 5G-6G Core.pdf
data\Understanding Mobile Traffic Patterns of Large Scale Cellular Towers in Urban Environment.pdf


100%|██████████| 34/34 [01:19<00:00,  2.35s/it]


Agora, vamos montar o Splitter, que fará a separação dos arquivos em **chunks**. Vamos usasr o [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter).

Os chunks terão um tamanho de 1500 caracteres e com um overlap de 150, para evitar que assuntos sejam abruptamente separados.

<img src="images/chunking.jpg" height="600px">

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


# O splitter fará o "fatiamento"
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)

Agora, vamos construir o vetor a partir dos chunks usando o [chromadb](https://www.trychroma.com/), um vectordb que vai armazenar os vetores criados a partir dos **embeddings** dos chunks.

In [11]:
import chromadb
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)


# Esses embeddings podem ser testados em casos específicos
# Para mais informações: https://www.sbert.net/
ef = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
#ef = embedding_functions.SentenceTransformerEmbeddingFunction('multi-qa-mpnet-base-dot-v1')
#ef = embedding_functions.SentenceTransformerEmbeddingFunction('multi-qa-mpnet-base-dot-v1')
#ef = SentenceTransformerEmbeddings(model_name='distiluse-base-multilingual-cased-v1')

In [12]:
from langchain.vectorstores import Chroma


# Onde o vectorDB será salvo
persist_directory = 'chroma_full/'

# Montagem
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=ef,
    persist_directory=persist_directory
)

In [30]:
# Fazendo um teste simples de retrieval
docs = vectordb.similarity_search('What is Nnwdaf_AnalyticsSubscriptionservice?')

In [83]:
# print results
print(docs[0].page_content)

individual sessions. The UPF processes and forwards user data
and is controlled by the SMF. In addition, the UPF connects to
external IP networks to act as anchor points, hiding mobility.
The Uniﬁed Data Management Function (UDM) accesses user
subscription data stored in the Uniﬁed Data Repository (UDR),a database containing network/user policies and associated
data. Finally, the Authentication Server Function provides au-
thentication services for a speciﬁc device, utilizing credentials
from the UDM [12].
As an underlying function solely responsible for data ana-
lytics and network learning, the NWDAF represents operator-
managed network analytics as a logical function [2]. The
NWDAF provides slice-speciﬁc network data analytics to any
given NF. As well, the NWDAF provides network analytics
information to NFs on a network slice instance level. The
function also notiﬁes NFs with slice-speciﬁc network status
analytic information for any that are subscribed to it. NFs may
also collect ne

# Configuração do prompt

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you can't make a answer with context, just say that you don't know, don't try to make up an answer. Give the document name from where the information is extracted
{context}
Question: {question}
Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Método de extração

Existem [diferentes formas](https://python.langchain.com/docs/modules/data_connection/retrievers/) de fazer a extração a partir da base de conhecimento. Aqui, usaremos um `LLMChainExtractor` que vai extrair dos vetores somente as parcelas mais úteis de cada um.

O `search_type='mmr'` garante um pouco mais de diversidade nas parcelas extraídas

In [78]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Wrap our vectorstore
compressor = LLMChainExtractor.from_llm(llm=flan_t5_model.to_langchain())  # uses an LLMChain to extract from each document only the statements that are relevant to the query.

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type='mmr')  # Retriever method
)

# Montagem da Chain

In [79]:
# Run chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm=flan_t5_model.to_langchain(),  # O modelo usado
#     retriever=vectordb.as_retriever(),  # O retriever usado
#     return_source_documents=True,
#     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},  # O prompt
# )

qa_chain = RetrievalQA.from_chain_type(
    llm=flan_t5_model.to_langchain(),  # O modelo usado
    retriever=compression_retriever,  # O retriever usado
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},  # O prompt
)

In [80]:
question = "What are the main types of open source databases used in 5G infrastructure?"
result = qa_chain({"query": question})

print('\n\n')
print(result["result"])  # Printando a saída
print('\n')

for source in result['source_documents']:  # Como os documentos base estão na resposta, podemos saber exatamente a partir de qual documento e página as respostas estão sendo geradas
    print ('Source: ', source.metadata['source'].split('/')[-1])
    print('Page: ', source.metadata['page'])
    print('\n')




Hadoop Distribute File System (HDFS)


Source:  data\Hierarchical, virtualised and distributed intelligence 5G architecture for low-latency and secure applications.pdf
Page:  3


Source:  data\Towards Supporting Intelligence in 5G-6G Core.pdf
Page:  1


Source:  data\Deep learning in mobile and wireless networking - A survey.pdf
Page:  20


Source:  data\Hierarchical, virtualised and distributed intelligence 5G architecture for low-latency and secure applications.pdf
Page:  4


